In [1]:
import textwrap

In [2]:
#Harcoded hash values for DES algorithm
parity_bit_drop_hash = [
    57, 49, 41, 33, 25, 17,  9,
      1, 58, 50, 42, 34, 26, 18,
    10,  2, 59, 51, 43, 35, 27,
    19, 11,  3, 60, 52, 44, 36,
    63, 55, 47, 39, 31, 23, 15,
      7, 62, 54, 46, 38, 30, 22,
    14,  6, 61, 53, 45, 37, 29,
    21, 13,  5, 28, 20, 12,  4
]
#inverse initial permutation in slides
final_permutation = [
    40,  8, 48, 16, 56, 24, 64, 32,
    39,  7, 47, 15, 55, 23, 63, 31,
    38,  6, 46, 14, 54, 22, 62, 30,
    37,  5, 45, 13, 53, 21, 61, 29,
    36,  4, 44, 12, 52, 20, 60, 28,
    35,  3, 43, 11, 51, 19, 59, 27,
    34,  2, 42, 10, 50, 18, 58, 26,
    33,  1, 41,  9, 49, 17, 57, 25
]
initial_permutation = [
    58, 50, 42, 34, 26, 18, 10,  2,
    60, 52, 44, 36, 28, 20, 12,  4,
    62, 54, 46, 38, 30, 22, 14,  6,
    64, 56, 48, 40, 32, 24, 16,  8,
    57, 49, 41, 33, 25, 17,  9,  1,
    59, 51, 43, 35, 27, 19, 11,  3,
    61, 53, 45, 37, 29, 21, 13,  5,
    63, 55, 47, 39, 31, 23, 15,  7
]
expansion_hash = [
    32,  1,  2,  3,  4,  5,
      4,  5,  6,  7,  8,  9,
      8,  9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32,  1
]
p_box = [
    16,  7, 20, 21, 29, 12, 28, 17,
      1, 15, 23, 26,  5, 18, 31, 10,
      2,  8, 24, 14, 32, 27,  3,  9,
    19, 13, 30,  6, 22, 11,  4, 25
]
s_boxes = {
    1: [
        [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
        [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
        [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
        [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
    ],
    2: [
        [15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
        [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
        [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
        [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]
    ],
    3: [
        [10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
        [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
        [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
        [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]
    ],
    4: [
        [ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
        [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
        [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
        [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]
    ],
    5: [
        [ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
        [14, 11,  2, 12,  4,  7, 13,  1,  5,  0,  3,  9,  8,  6, 15, 10],
        [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
        [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]
    ],
    6: [
        [12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
        [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
        [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
        [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]
    ],
    7: [
        [ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
        [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
        [ 1,  4, 11, 13, 12,  3,  7, 15, 14,  5,  2,  8,  6,  9,  0, 10],
        [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8]
    ],
    8: [
        [13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
        [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
        [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
        [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]
    ]
}
key_compression_hash = [
    14, 17, 11, 24,  1,  5,
      3, 28, 15,  6, 21, 10,
    23, 19, 12,  4, 26,  8,
    16,  7, 27, 20, 13,  2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]

In [3]:
import textwrap

class DES_algorithm:
    def __init__(self, key):
        if len(key) != 8:
            raise ValueError("Key must be 8 characters long")
        
        key_in_bits = []
        for char in key:
            if ord(char) > 255:
                raise ValueError("Key must be ASCII characters")
            ascii_val = ord(char)
            binary_val = format(ascii_val, '08b')
            key_in_bits += [int(bit) for bit in binary_val]
        
        self.key = key_in_bits
        self.subkeys = self.generate_subkeys(key_in_bits)

    def xor(self, x1, x2):
        return x1 ^ x2

    def left_shift(self, bits, num_shifts):
        return bits[num_shifts:] + bits[:num_shifts]

    def expansion_p_box(self, right_32_bits):
        return [right_32_bits[i - 1] for i in expansion_hash]

    def straight_p_box(self, right_32_bits):
        return [right_32_bits[i - 1] for i in p_box]

    def s_box_transformation(self, right_48_bits):
        s_box_output_32_bits = []
        start_counter = 0
        end_counter = 6
        curr_block = 1
        for _ in range(8):
            six_bits = right_48_bits[start_counter:end_counter]
            row = int(str(six_bits[0]) + str(six_bits[5]), 2)
            col = int("".join(str(b) for b in six_bits[1:5]), 2)
            s_box_value = s_boxes[curr_block][row][col]
            s_box_bits = format(s_box_value, '04b')
            s_box_output_32_bits += [int(bit) for bit in s_box_bits]
            start_counter += 6
            end_counter += 6
            curr_block += 1
        return s_box_output_32_bits

    def F_box(self, right_32_bits, subkey):
        expanded = self.expansion_p_box(right_32_bits)
        xored = [self.xor(expanded[i], subkey[i]) for i in range(48)]
        s_out = self.s_box_transformation(xored)
        return self.straight_p_box(s_out)

    def initial_permutation(self, bits):
        return [bits[i - 1] for i in initial_permutation]

    def inverse_initial_permutation(self, bits):
        return [bits[i - 1] for i in final_permutation]

    def parity_drop(self, bits):
        return [bits[i - 1] for i in parity_bit_drop_hash]

    def generate_subkeys(self, key):
        key_56 = self.parity_drop(key)
        subkeys = []
        left_key = key_56[:28]
        right_key = key_56[28:]
        shifting_hash = [1, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1]
        for i in range(16):
            left_key = self.left_shift(left_key, shifting_hash[i])
            right_key = self.left_shift(right_key, shifting_hash[i])
            combined = left_key + right_key
            subkey = [combined[j - 1] for j in key_compression_hash]
            subkeys.append(subkey)
        return subkeys

    def transform_input(self, text, block_size=8):
        binary_str = "".join(format(ord(c), '08b') for c in text)
        blocks = textwrap.wrap(binary_str, block_size * 8)
        if blocks:
            last_len = len(blocks[-1])
            needed = block_size * 8 - last_len
            if 0 < needed < block_size * 8:
                blocks[-1] += "0" * needed
        return blocks

    def single_block_encryption(self, block, subkeys):
        permuted = self.initial_permutation(block)
        left = permuted[:32]
        right = permuted[32:]
        round1_output = None
        round14_output = None
        for i in range(16):
            f_out = self.F_box(right, subkeys[i])
            new_left = right
            new_right = [self.xor(left[k], f_out[k]) for k in range(32)]
            left, right = new_left, new_right

            if i == 0:
                round1_output = (left[:], right[:])
            if i == 13:
                round14_output = (left[:], right[:])

        swapped = right + left
        final_block = self.inverse_initial_permutation(swapped)
        return final_block, round1_output, round14_output

    def encryption(self, plaintext):
        blocks = self.transform_input(plaintext)
        encrypted = []
        for b_str in blocks:
            bits = [int(x) for x in b_str]
            final_block, _, _ = self.single_block_encryption(bits, self.subkeys)
            encrypted.append(final_block)
        return "".join(str(b) for block in encrypted for b in block)

    def single_block_decryption(self, block, subkeys):
        permuted = self.initial_permutation(block)
        left = permuted[:32]
        right = permuted[32:]
        round2_output = None
        round15_output = None

        for i in range(16):
            f_out = self.F_box(right, subkeys[15 - i])
            new_left = right
            new_right = [self.xor(left[k], f_out[k]) for k in range(32)]
            left, right = new_left, new_right

            if i == 1:
                round2_output = (right[:], left[:])
            if i == 14:
                round15_output = (right[:], left[:])

        swapped = right + left
        final_block = self.inverse_initial_permutation(swapped)
        return final_block, round2_output, round15_output


    def decryption(self, ciphertext_bits):
        size = 64
        blocks = textwrap.wrap(ciphertext_bits, size)
        decrypted = []
        for b_str in blocks:
            bits = [int(x) for x in b_str]
            final_block, _, _ = self.single_block_decryption(bits, self.subkeys)
            decrypted.append(final_block)
        return "".join(str(b) for block in decrypted for b in block)

    @staticmethod
    def bits_to_text(bits):
        chars = []
        # Process 8 bits at a time:
        for i in range(0, len(bits), 8):
            byte_str = "".join(str(b) for b in bits[i:i+8])
            byte_val = int(byte_str, 2)
            chars.append(chr(byte_val))
        return "".join(chars)


In [4]:

if __name__ == "__main__":
    # 1) Create the DES object with your 8-character key
    key_str = "DES_KEY!"
    des = DES_algorithm(key_str)

    plaintexts = [
        "AnArDES!",
        "20712091",
        "cse350A2"
    ]

    for pt in plaintexts:
        print("\n===================================")
        print("Plaintext:", pt)

        blocks = des.transform_input(pt)
        first_block_bits = [int(b) for b in blocks[0]]
        enc_block_bits, enc_r1, enc_r14 = des.single_block_encryption(first_block_bits, des.subkeys)

        # Convert entire plaintext to ciphertext
        ciphertext_bits = des.encryption(pt)
        # Decrypt entire ciphertext
        decrypted_bits  = des.decryption(ciphertext_bits)
        recovered_text = DES_algorithm.bits_to_text(decrypted_bits).rstrip('\x00')

        print("Ciphertext (first 64 bits):", ciphertext_bits[:64], "...")
        print("Recovered text:", recovered_text)
        print("Match? ", ("YES" if recovered_text == pt else "NO"))

        # 3b & 3c) Compare intermediate rounds:
        dec_block_bits, dec_r2, dec_r15 = des.single_block_decryption(enc_block_bits, des.subkeys)

        print("\n(b) Checking 1st encryption round vs 15th decryption round:")
        if enc_r1 == dec_r15:
            print("    They match! =>")
            print("    Enc Round1:", enc_r1, "| Dec Round15:", dec_r15)
            # Print ASCII representation (4 chars for each 32-bit half)
            enc_r1_left_ascii  = des.bits_to_text(enc_r1[0])
            enc_r1_right_ascii = des.bits_to_text(enc_r1[1])
            dec_r15_left_ascii = des.bits_to_text(dec_r15[0])
            dec_r15_right_ascii = des.bits_to_text(dec_r15[1])
            
            print("    [Enc Round1 Left ASCII]:", repr(enc_r1_left_ascii),
                  "| [Enc Round1 Right ASCII]:", repr(enc_r1_right_ascii))
            print("    [Dec Round15 Left ASCII]:", repr(dec_r15_left_ascii),
                  "| [Dec Round15 Right ASCII]:", repr(dec_r15_right_ascii))
        else:
            print("    Mismatch.\n    Enc Round1:", enc_r1,
                  "\n    Dec Round15:", dec_r15)

        print("\n(c) Checking 14th encryption round vs 2nd decryption round:")
        if enc_r14 == dec_r2:
            print("    They match! =>")
            print("    Enc Round1:", enc_r1, "| Dec Round15:", dec_r15)
            
            # Print ASCII representation    
            enc_r14_left_ascii  = des.bits_to_text(enc_r14[0])
            enc_r14_right_ascii = des.bits_to_text(enc_r14[1])
            dec_r2_left_ascii = des.bits_to_text(dec_r2[0])
            dec_r2_right_ascii = des.bits_to_text(dec_r2[1])
            print("    [Enc Round14 Left ASCII]:", repr(enc_r14_left_ascii),
                  "| [Enc Round14 Right ASCII]:", repr(enc_r14_right_ascii))
            print("    [Dec Round2 Left ASCII]:", repr(dec_r2_left_ascii),
                  "| [Dec Round2 Right ASCII]:", repr(dec_r2_right_ascii))
        else:
            print("    Mismatch.\n    Enc Round14:", enc_r14,
                  "\n    Dec Round2:", dec_r2)

        print("===================================")


Plaintext: AnArDES!
Ciphertext (first 64 bits): 0110110111000000111100110001011011011010011110100001100100100110 ...
Recovered text: AnArDES!
Match?  YES

(b) Checking 1st encryption round vs 15th decryption round:
    They match! =>
    Enc Round1: ([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0]) | Dec Round15: ([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0])
    [Enc Round1 Left ASCII]: '\x00\x8a\x02J' | [Enc Round1 Right ASCII]: '5\x99\x8f\x96'
    [Dec Round15 Left ASCII]: '\x00\x8a\x02J' | [Dec Round15 Right ASCII]: '5\x99\x8f\x96'

(c) Checking 14th encryption round vs 2nd decryption round:
    They match! =>
    Enc Round1: ([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0,